In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Automatic Setting of Execution Order

By default, subsystems of a Group are executed in the same order as they were added to that Group.
Depending on the data connections between those subsystems, that order could result in some subsystems being executed *before* their inputs have been 
updated.  This can result in a performance impact by increasing the number of solver iterations required to achieve convergence.

In come cases it's obvious what the order should be and you can simply declare the subsystems in the correct order.  However, as models evolve and 
become more complex, it becomes more likely that out of order execution may occur.  One way to prevent this from happening is to set the `auto_order` option on the Group.  This will cause OpenMDAO to set the Group's subsystem execution order based on the dependency graph of that Group.  Note that if connections within a Group result in some subsystems forming a cycle in the graph, then OpenMDAO will not attempt to reorder the cycle.  It will, 
however, order the cycle properly with respect to any subsystems outside of the cycle.  Below is a simple example where OpenMDAO will automatically
reorder the subsystems of group `G1` based on their data dependencies.

In [ ]:
import openmdao.api as om

class SimpleComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x')
        self.add_output('y')
        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        print(f"running {self.name}")
        outputs['y'] = 2.0*inputs['x']


p = om.Problem()
model = p.model
# tell OpenMDAO to auto-order the components in G1
G1 = model.add_subsystem('G1', om.Group(auto_order=True))
G1.add_subsystem('C2', SimpleComp())
G1.add_subsystem('C1', SimpleComp())
G1.add_subsystem('C3', SimpleComp())
G1.connect('C1.y', 'C2.x')
G1.connect('C2.y', 'C3.x')


p.setup()
p.run_model()

In [ ]:
order = [s.name for s in G1._subsystems_myproc]
assert order == ['C1', 'C2', 'C3'], f"order should be ['C1', 'C2', 'C3'] but is {order}"

The components in G1 were declared in the order ['C2', 'C1', 'C3'], but they should execute in the order ['C1', 'C2', 'C3'] based on how they are connected to each other.